<a href="https://colab.research.google.com/github/ArrasyEdlyn/edlyn-dirc/blob/main/Copy_of_Project_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install faiss-cpu sentence-transformers

In [ ]:
"""https://huggingface.co/deepset/roberta-base-squad2"""
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
"""https://huggingface.co/deepset/roberta-base-squad2"""
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'At what temperature does water boil at sea level?',
    'context': 'Water boils at 100 degrees Celsius at sea level. However, at higher altitudes, due to lower atmospheric pressure, water boils at a lower temperature. This principle is important in cooking and scientific experiments conducted at different elevations.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Device set to use cuda:0
Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


In [ ]:
res # score == F1 Score: it's computed on the individual words in the prediction vs the true words provided in context

{'score': 0.9166737794876099,
 'start': 15,
 'end': 34,
 'answer': '100 degrees Celsius'}

In [ ]:
model

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

dataset = load_dataset("deepset/covid_qa_deepset")

df = pd.DataFrame(dataset["train"])



In [ ]:
# Remove missing values
df = df.dropna()

# Extract the answer text from the "answers" dictionary
df["answer"] = df["answers"].apply(lambda x: x["text"][0] if len(x["text"]) > 0 else "")

# Convert text to lowercase (optional)
df["question"] = df["question"].str.lower()
df["context"] = df["context"].str.lower()
df["answer"] = df["answer"].str.lower()

print(df[["question", "context", "answer"]].head())

                                            question  \
0  what is the main cause of hiv-1 infection in c...   
1  what plays the crucial role in the mother to c...   
2  how many children were infected by hiv-1 in 20...   
3  what is the role of c-c motif chemokine ligand...   
4        what is dc-genr and where is  it expressed?   

                                             context  \
0  functional genetic variants in dc-signr are as...   
1  functional genetic variants in dc-signr are as...   
2  functional genetic variants in dc-signr are as...   
3  functional genetic variants in dc-signr are as...   
4  functional genetic variants in dc-signr are as...   

                                              answer  
0  mother-to-child transmission (mtct) is the mai...  
1  dc-signr plays a crucial role in mtct of hiv-1...  
2  more than 400,000 children were infected world...  
3  high copy numbers of ccl3l1, a potent hiv-1 su...  
4  dendritic cell-specific icam-grabbing non-inte..

In [ ]:
# Load sentence transformer for embeddings
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Convert context texts into embeddings
context_texts = df["context"].tolist()
context_embeddings = embed_model.encode(context_texts, convert_to_numpy=True)

In [ ]:
print(context_embeddings)

[[-0.07105713 -0.04538158  0.03040162 ... -0.02389823  0.03788241
  -0.03420835]
 [-0.07105713 -0.04538158  0.03040162 ... -0.02389823  0.03788241
  -0.03420835]
 [-0.07105713 -0.04538158  0.03040162 ... -0.02389823  0.03788241
  -0.03420835]
 ...
 [ 0.02422593  0.06549928  0.02575275 ... -0.12366255 -0.03642359
   0.00460077]
 [ 0.02422593  0.06549928  0.02575275 ... -0.12366255 -0.03642359
   0.00460077]
 [ 0.02422593  0.06549928  0.02575275 ... -0.12366255 -0.03642359
   0.00460077]]


In [ ]:
from sklearn.model_selection import train_test_split

# Split into training and validation only (e.g., 90% train, 10% validation)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

print(f"Training Set Size: {len(train_df)}")
print(f"Validation Set Size: {len(val_df)}")
train_df.head()
val_df.head()

Training Set Size: 1817
Validation Set Size: 202


,document_id,context,question,is_impossible,id,answers,answer
1556,2504,respiratory viral infections in exacerbation o...,what have the authors reviewed?,False,3858,{'text': ['the prevalence of different respira...,the prevalence of different respiratory viral ...
526,2440,optimization method for forecasting confirmed ...,how does the proposed fpassa start?,False,4432,{'text': ['by receiving the historical COVID-1...,by receiving the historical covid-19 dataset.
393,1604,emergent severe acute respiratory distress syn...,what is the mean time from onset of symptoms t...,False,3245,"{'text': ['5 days'], 'answer_start': [8958]}",5 days
1789,2642,first cases of coronavirus disease 2019 (covid...,what was the clinical evolution of the hospita...,False,3828,{'text': ['All hospitalised cases had a benign...,all hospitalised cases had a benign clinical e...
433,1632,metabolic engineering of escherichia coli into...,what characteristics does glycosylation have o...,False,5300,"{'text': ['solubility, stability and bio-activ...","solubility, stability and bio-activity"


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "deepset/roberta-base-squad2"  # Use roberta-base instead of larger models
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)



In [ ]:
def preprocess(examples):
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_offsets_mapping=True,  # Helps map token positions to character positions
    )

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(inputs["offset_mapping"]):
        answers = examples["answers"][i]
        if len(answers["text"]) > 0:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Find token start & end indices
            start_token = end_token = 0
            for idx, (start, end) in enumerate(offsets):
                if start_char >= start and start_char < end:
                    start_token = idx
                if end_char > start and end_char <= end:
                    end_token = idx

            start_positions.append(start_token)
            end_positions.append(end_token)
        else:
            start_positions.append(0)
            end_positions.append(0)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    # Remove offset mapping after processing
    inputs.pop("offset_mapping")

    return inputs


In [ ]:
dimension = context_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)  # L2 distance index
faiss_index.add(context_embeddings)  # Add embeddings to FAISS

# Save index (optional)
faiss.write_index(faiss_index, "faiss_index.bin")

print("FAISS index created with", len(context_texts), "contexts.")

FAISS index created with 2019 contexts.


In [ ]:
train_data = Dataset.from_pandas(train_df)
valid_data = Dataset.from_pandas(val_df)
# Apply preprocessing to dataset
train_data = train_data.map(preprocess, batched=True)
valid_data = valid_data.map(preprocess, batched=True)

Map:   0%|          | 0/1817 [00:00<?, ? examples/s]

Map:   0%|          | 0/202 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./qa_model",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=valid_data
)

# Start training
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.667153
2,No log,0.712957
3,0.493300,0.773682


TrainOutput(global_step=684, training_loss=0.4163253293399922, metrics={'train_runtime': 638.7119, 'train_samples_per_second': 8.534, 'train_steps_per_second': 1.071, 'total_flos': 1424328620967936.0, 'train_loss': 0.4163253293399922, 'epoch': 3.0})

In [ ]:
from transformers import pipeline

# Load the trained model into a QA pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

def retrieve_context(question, top_k=1):
    # Convert question to embedding
    question_embedding = embed_model.encode([question], convert_to_numpy=True)

    # Search FAISS index
    distances, indices = faiss_index.search(question_embedding, top_k)

    # Retrieve best-matching context (select top one)
    retrieved_context = context_texts[indices[0][0]]  # Take only the best match

    return retrieved_context  # Return single string

# Example Question
question = "where the pandemic place start?"
context = retrieve_context(question)  # Retrieve relevant context

# Get the answer
result = qa_pipeline(question=question, context=context)

# Print the result
print("Answer:", result["answer"])
print("Score:", result["score"])

Device set to use cuda:0


Answer: 191871919
Score: 0.008632649667561054


In [ ]:
# Load the QA pipeline with our trained model
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Provide a sample context
context = """Coronavirus disease (COVID-19) is an infectious disease caused by the SARS-CoV-2 virus.
Most people infected with the virus will experience mild to moderate respiratory illness and recover without requiring special treatment.
However, some will become seriously ill and require medical attention."""

questions = [
    "What causes COVID-19?",
    "How does COVID-19 spread?",
    "What are the symptoms of COVID-19?"
]

for q in questions:
    result = qa_pipeline(question=q, context=context)
    print(f"Q: {q} \nA: {result['answer']} (Score: {result['score']:.4f})\n")


In [ ]:
!pip install faiss-cpu sentence-transformers

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_name) # remember that model_name is deepset/roberta-base-squad2

In [ ]:
# Let's start training!
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=0.01,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=11,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


NameError: name 'tokenized_squad' is not defined